In [3]:
# to calculate paralogue family average dT and compare species to show whether the same family would divergence
# for both species on pairwise comparison.
suppressPackageStartupMessages({
    require(igraph)
    library(dplyr)
    library(ggvenn)
    library(tidyr)
    library(ggpubr)
    library(rstatix)
    library(stringr)
})

In [4]:
# get gene relationships
duplicate_pairs <- readRDS("Combined.SSD_WGD.rds")

orthogroups <- read.delim('/mnt/data01/yuanzhen/01.Vertebrate_cell_evo/02.gene_relationships/run4/results/Ortho_pipeline/OrthoFinder/Orthogroups/Orthogroups.tsv')
# *** at least one copy for at least one species
orthogroups <- orthogroups %>% select(c('Orthogroup', 'Pmar', 'Pvit', 'Mmus', 'Hsap'))  %>% 
    filter(Pmar != '' | Pvit != ''  | Mmus != '' | Hsap != '')

# calculate number of genes by calcualting commas in it
count_commas <- function(x) {
  sapply(gregexpr(",", x), function(match) ifelse(match[1] == -1, 0, length(match)))
}
number_genes <- data.frame(apply(orthogroups, c(1,2), count_commas))

# retain orthogroups with only 5 copies max for each four species
orthogroups <- orthogroups[which(number_genes$Pmar <= 4 & number_genes$Pvit <= 4 & number_genes$Mmus <= 4 & number_genes$Hsap <= 4), ]

In [5]:
WGD_genes <- unique(c(duplicate_pairs[duplicate_pairs$type == 'WGD', 'dup1'], duplicate_pairs[duplicate_pairs$type == 'WGD', 'dup2']))
SSD_genes <- unique(c(duplicate_pairs[duplicate_pairs$type == 'SSD', 'dup1'], duplicate_pairs[duplicate_pairs$type == 'SSD', 'dup2']))
paralog_genes <- unique(c(duplicate_pairs[,'dup1'], duplicate_pairs[,'dup2']))



Paralog_orthogroups <- orthogroups[apply(orthogroups, 1, function(row) {
  any(sapply(row, function(cell) {
    sum(paralog_genes %in% unlist(strsplit(as.character(cell), ", ")))>=2
  }))
}), ]

In [8]:
exp_domain <- readRDS('exp_domain/combined.exp_tri_score.rds')
head(exp_domain)

,gene,cluster,species,species_gene
,<chr>,<chr>,<chr>,<chr>
1,ENSG00000188290,Astrocytes,Hsap,Hsap_ENSG00000188290
2,ENSG00000078808,Astrocytes,Hsap,Hsap_ENSG00000078808
3,ENSG00000160087,Astrocytes,Hsap,Hsap_ENSG00000160087
4,ENSG00000131584,Astrocytes,Hsap,Hsap_ENSG00000131584
5,ENSG00000127054,Astrocytes,Hsap,Hsap_ENSG00000127054
6,ENSG00000107404,Astrocytes,Hsap,Hsap_ENSG00000107404


In [9]:
get_gene_ID <- function(row, species){
    return(unlist(strsplit(as.character(row[[species]]), ", ")))
}

calculate_pairwise_dT_for_gene_pairs <- function(genes, exp_domain, species){
    exp_domain <- exp_domain[exp_domain$species == species, ] 
    
    if (length(genes) < 2){
        dTs <- NA
    } else {
        gene_pairs <- combn(genes, 2)
        dTs <- unlist(lapply(1:ncol(gene_pairs), FUN = function(x){
            pairs <- gene_pairs[,x]
            # Either copy expressed
            Either <- unique(exp_domain[exp_domain$gene %in% pairs, 'cluster'])
            # Both copy expressed
            Both <- intersect(exp_domain[exp_domain$gene %in% pairs[1], 'cluster'], 
                              exp_domain[exp_domain$gene %in% pairs[2], 'cluster'])
            dT <- (length(Either)-length(Both))/length(Either)
            # very important!!!! for example TBR1, TBX21, and EOMES case for human
            # TBX21 and EOMES not expressed in brain, the dT = NaN
            # in this case, set dT as 1
            if (is.na(dT)){
                dT = 1
            }
            return(dT)
        }))
        # average for each gene family multiple pairwise comparsion
        dTs <- sum(dTs)/length(dTs)  
    }
    return(dTs)
}

In [10]:
get_results <- function(orthogroup){
    results <- Reduce(rbind, lapply(1:nrow(orthogroup), FUN = function(x){
        row = orthogroup[x, ]
        # get gene IDs for each species
        Hsap = get_gene_ID(row, 'Hsap')
        Mmus = get_gene_ID(row, 'Mmus')
        Pvit = get_gene_ID(row, 'Pvit')
        Pmar = get_gene_ID(row, 'Pmar')
        
        Hsap_dTs <- calculate_pairwise_dT_for_gene_pairs(Hsap, exp_domain, 'Hsap')
        Mmus_dTs <- calculate_pairwise_dT_for_gene_pairs(Mmus, exp_domain, 'Mmus')
        Pvit_dTs <- calculate_pairwise_dT_for_gene_pairs(Pvit, exp_domain, 'Pvit')
        Pmar_dTs <- calculate_pairwise_dT_for_gene_pairs(Pmar, exp_domain, 'Pmar')
        
        info <- c(unlist(row[['Orthogroup']]), Hsap_dTs, Mmus_dTs, Pvit_dTs, Pmar_dTs)
        return(info)
    }))
    results <- as.data.frame(results)
    colnames(results) <- c('Orthogroup', 'Hsap', 'Mmus', 'Pvit', 'Pmar')
    return(results)
}

In [11]:
dTs_paralog_orthogroups <- get_results(Paralog_orthogroups)
dTs_paralog_orthogroups[dTs_paralog_orthogroups == "NaN"] <- NA

In [12]:
get_plots <- function(species1, species2){
    tmp = dTs_paralog_orthogroups[, c(species1, species2)]
    tmp = tmp[complete.cases(tmp), ]
    tmp[,species1] <- as.double(tmp[,species1])
    tmp[,species2] <- as.double(tmp[,species2])
    scatter_plot <- ggscatter(tmp, x = species1, y = species2) + 
        stat_cor(method = "pearson") +  theme_minimal() 
    return(scatter_plot)
}

In [13]:
p1 <- get_plots('Hsap', 'Mmus')
p2 <- get_plots('Hsap', 'Pvit')
p3 <- get_plots('Hsap', 'Pmar')
p4 <- get_plots('Mmus', 'Pvit')
p5 <- get_plots('Mmus', 'Pmar')
p6 <- get_plots('Pvit', 'Pmar')

ggsave('averged.dT.Hsap_vs_Mmus.pdf', p1, width = 5, height = 5)
ggsave('averged.dT.Hsap_vs_Pvit.pdf', p2, width = 5, height = 5)
ggsave('averged.dT.Hsap_vs_Pmar.pdf', p3, width = 5, height = 5)
ggsave('averged.dT.Mmus_vs_Pvit.pdf', p4, width = 5, height = 5)
ggsave('averged.dT.Mmus_vs_Pmar.pdf', p5, width = 5, height = 5)
ggsave('averged.dT.Pvit_vs_Pmar.pdf', p6, width = 5, height = 5)

In [14]:
save.image('s3.calculate_divergence_across_species.RData')

In [ ]:
# test cases to show, pls ignore below!

In [2]:
load('s3.calculate_divergence_across_species.RData')

In [22]:
# potential groups for searching cases
library(entropy)
test <- dTs_paralog_orthogroups[complete.cases(dTs_paralog_orthogroups), ]

In [15]:
dTs_paralog_orthogroups[dTs_paralog_orthogroups$Orthogroup == 'OG0000434',]

,Orthogroup,Hsap,Mmus,Pvit,Pmar
,<chr>,<chr>,<chr>,<chr>,<chr>
X.111,OG0000434,0.0992647058823529,0.226538568643832,0.507407407407407,0.441544117647059
